In [2]:
import keras
from keras import layers
from keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape
from keras.layers import BatchNormalization
from keras.models import Model
from keras.datasets import mnist
from keras.losses import binary_crossentropy
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt

original_dim = 28 * 28
intermediate_dim = 64
latent_dim = 2

inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)


from keras import backend as K

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0, stddev=0.1)
    return z_mean + K.exp(z_log_sigma)*epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])


# Create encoder
encoder = keras.Model(inputs,[z_mean, z_log_sigma, z], name='encoder')

#Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')



(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
x_train = x_train.reshape(len(x_train), np.prod(x_train.shape[1:]))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))


outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')


(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

vae.fit(x_train, x_train,
        epochs=100,
        batch_size=32,
        validation_data=(x_test, x_test))


x_test_encoded = encoder.predict(x_test, batch_size=256)
plt.figure(figsize=(6,6))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test)
plt.colorbar()
plt.show()

Epoch 1/100
1875/1875 [==============================] - 4s 2ms/step - loss: 187.9203 - val_loss: 168.8217
Epoch 2/100
1875/1875 [==============================] - 4s 2ms/step - loss: 166.9245 - val_loss: 164.2847
Epoch 3/100
1875/1875 [==============================] - 4s 2ms/step - loss: 162.9566 - val_loss: 161.6497
Epoch 4/100
1875/1875 [==============================] - 4s 2ms/step - loss: 160.4722 - val_loss: 159.4939
Epoch 5/100
1875/1875 [==============================] - 4s 2ms/step - loss: 158.9344 - val_loss: 158.4329
Epoch 6/100
1875/1875 [==============================] - 3s 2ms/step - loss: 157.8224 - val_loss: 157.4061
Epoch 7/100
1875/1875 [==============================] - 3s 2ms/step - loss: 156.8652 - val_loss: 156.7211
Epoch 8/100
1875/1875 [==============================] - 3s 2ms/step - loss: 156.0978 - val_loss: 156.1834
Epoch 9/100
1875/1875 [==============================] - 3s 2ms/step - loss: 155.4553 - val_loss: 155.8069
Epoch 10/100
1875/1875 [=============

Epoch 77/100
1875/1875 [==============================] - 5s 3ms/step - loss: 145.9350 - val_loss: 148.5657
Epoch 78/100
1875/1875 [==============================] - 5s 3ms/step - loss: 145.8987 - val_loss: 148.3852
Epoch 79/100
1875/1875 [==============================] - 5s 3ms/step - loss: 145.8746 - val_loss: 148.3918
Epoch 80/100
1875/1875 [==============================] - 5s 3ms/step - loss: 145.8381 - val_loss: 148.3509
Epoch 81/100
1875/1875 [==============================] - 6s 3ms/step - loss: 145.7948 - val_loss: 148.3186
Epoch 82/100
1875/1875 [==============================] - 5s 3ms/step - loss: 145.7915 - val_loss: 148.2592
Epoch 83/100
1875/1875 [==============================] - 5s 3ms/step - loss: 145.7301 - val_loss: 148.2926
Epoch 84/100
1875/1875 [==============================] - 6s 3ms/step - loss: 145.7074 - val_loss: 148.1311
Epoch 85/100
1875/1875 [==============================] - 5s 3ms/step - loss: 145.6668 - val_loss: 148.2222
Epoch 86/100
1875/1875 [====

TypeError: list indices must be integers or slices, not tuple

<Figure size 432x432 with 0 Axes>